# Validation

We primarily validate tax model input datasets by comparing against the IRS Survey of Incomes data releases. For each data year, we have access to between 1,000 and 2,000 individual statistical targets (e.g. "total taxable Social Security for filers with AGI between 30k and 40k"). For each dataset produced by this repo, we can attempt to reproduce these statistics.

There are valid reasons why datasets might not be able to reproduce SOI statistics:

1. The PUF sample size is too low (e.g. "estate losses" are in the tens of thousands for some AGI breakdowns).
2. The PUF is missing granular tax form information (e.g. some capital gains tax details that form the totals are not in the PUF data).
3. Datasets that are for a tax model produce different projected tax output variables than the PUF (e.g. Tax-Calculator or PolicyEngine might have a different EITC value than is reported in the PUF data).

Regardless, it's still useful to know because we should be able to reproduce most of the SOI statistics.

## Measuring quality

One way to measure the quality of fit against SOI targets might be to just take the mean relative deviation. However, the SOI statistics that fall under category (1) (and to some extent the others) tend to blow up this mean. Those SOI statistics might not be particularly important, and we want the quality of fit indicator to have some useful informational quality, so this is not ideal.

Instead, this validation exercise marks an SOI statistic as "OK" if the relative deviation is less than 5 percent, or if the absolute deviation is less than 1 million for filer count statistics and 1 billion for aggregate statistics.

With that definition, we can report to the percentage of SOI statistics (in the relevant year) that are "OK" for each dataset below.

In [1]:
from tax_microdata_benchmarking.utils.soi_replication import *
from tax_microdata_benchmarking.storage import STORAGE_FOLDER
from tax_microdata_benchmarking.datasets import *
import pandas as pd

INPUTS = STORAGE_FOLDER / "input"
OUTPUTS = STORAGE_FOLDER / "output"

puf_2015 = pd.read_csv(INPUTS / "puf_2015.csv")
tc_puf_2015 = pd.read_csv(OUTPUTS / "tc_puf_2015.csv")

soi_from_puf_2015 = compare_soi_replication_to_soi(
    puf_to_soi(puf_2015, 2015), 2015
)
soi_from_pe_puf_2015 = compare_soi_replication_to_soi(
    pe_to_soi(PUF_2015, 2015), 2015
)
soi_from_tc_puf_2015 = compare_soi_replication_to_soi(
    tc_to_soi(tc_puf_2015, 2015), 2015
)


def soi_statistic_passes_quality_test(df):
    # Relative error lower than this => OK
    RELATIVE_ERROR_THRESHOLD = 0.05

    # Absolute error lower than this for filer counts => OK
    COUNT_ABSOLUTE_ERROR_THRESHOLD = 1e6

    # Absolute error lower than this for aggregates => OK
    AGGREGATE_ABSOLUTE_ERROR_THRESHOLD = 1e9

    relative_error_ok = (
        df["Absolute relative error"] < RELATIVE_ERROR_THRESHOLD
    )
    absolute_error_threshold = np.where(
        df.Count,
        COUNT_ABSOLUTE_ERROR_THRESHOLD,
        AGGREGATE_ABSOLUTE_ERROR_THRESHOLD,
    )
    absolute_error_ok = df["Absolute error"] < absolute_error_threshold

    return relative_error_ok | absolute_error_ok


# 2021 datasets

puf_2021 = pd.read_csv(OUTPUTS / "puf_2021.csv")
tc_puf_2021 = pd.read_csv(OUTPUTS / "tc_puf_2021.csv")
tmd_2021 = pd.read_csv(OUTPUTS / "tmd_2021.csv")

soi_from_puf_2021 = compare_soi_replication_to_soi(
    puf_to_soi(puf_2021, 2021), 2021
)
soi_from_pe_puf_2021 = compare_soi_replication_to_soi(
    pe_to_soi(PUF_2021, 2021), 2021
)
soi_from_tc_puf_2021 = compare_soi_replication_to_soi(
    tc_to_soi(tc_puf_2021, 2021), 2021
)
soi_from_tmd_2021 = compare_soi_replication_to_soi(
    tc_to_soi(tmd_2021, 2021), 2021
)

dataset_soi_comparisons = [
    soi_from_puf_2015,
    soi_from_pe_puf_2015,
    soi_from_tc_puf_2015,
    soi_from_puf_2021,
    soi_from_pe_puf_2021,
    soi_from_tc_puf_2021,
    soi_from_tmd_2021,
]

for dataset in dataset_soi_comparisons:
    dataset["OK"] = soi_statistic_passes_quality_test(dataset)

dataset_names = [
    "PUF (2015)",
    "PE PUF (2015)",
    "TC PUF (2015)",
    "PUF (2021)",
    "PE PUF (2021)",
    "TC PUF (2021)",
    "TMD (2021)",
]

comparison_df = pd.DataFrame(
    {
        "Dataset": dataset_names,
        "SOI match score": [
            (df["OK"].mean() * 100).round(1) for df in dataset_soi_comparisons
        ],
    }
)

comparison_df

,Dataset,SOI match score
0,PUF (2015),95.2
1,PE PUF (2015),80.5
2,TC PUF (2015),89.4
3,PUF (2021),63.4
4,PE PUF (2021),70.7
5,TC PUF (2021),68.6
6,TMD (2021),75.6


Note that the pure PUF-derived datasets have lower scores than the PUF with reported tax output values. This is because the tax models (both of them) produced different estimates for tax variables, including the adjusted gross income which is used to bracket out some of the SOI statistics. This can mean that even if, for example, tax-exempt pension income is simply copied directly into the dataset, if the tax model produces different adjusted gross incomes for records then the tax-exempt pension income SOI statistics by AGI might be different than in the SOI releases.

## SOI score by variable

We can break out this score further, by variable.

In [2]:
soi_from_tmd_2021.to_csv(OUTPUTS / "soi_from_puf_tmd_2021.csv", index=False)

In [3]:
score_by_dataset = pd.DataFrame(
    {
        dataset_name: (dataset.groupby("Variable").OK.mean() * 100).round(1)
        for dataset_name, dataset in zip(
            dataset_names, dataset_soi_comparisons
        )
    }
).fillna(
    100
)  # Fillna because some variables aren't in the 2021 SOI releases.
score_by_dataset.sort_values("TMD (2021)")

,PUF (2015),PE PUF (2015),TC PUF (2015),PUF (2021),PE PUF (2021),TC PUF (2021),TMD (2021)
Variable,,,,,,,
income_tax_after_credits,100.0,40.0,72.5,42.7,31.7,35.4,39.0
rent_and_royalty_net_income,88.1,92.9,47.6,65.3,80.6,54.2,54.2
unemployment_compensation,100.0,100.0,100.0,65.3,58.3,56.9,61.1
state_and_local_tax_deductions,54.2,52.1,56.2,41.7,52.1,60.4,62.5
interest_paid_deductions,100.0,89.6,100.0,52.1,64.6,64.6,62.5
business_net_profits,100.0,95.2,97.6,55.6,52.8,55.6,63.9
capital_gains_gross,97.6,66.7,66.7,63.9,59.7,56.9,66.7
total_social_security,97.6,90.5,97.6,61.1,81.9,73.6,66.7
rent_and_royalty_net_losses,83.3,88.1,61.9,73.6,80.6,68.1,68.1


In [4]:
# What happened over reweighting?

from IPython.display import Markdown

soi_from_tmd_2021["Improved"] = (
    soi_from_tmd_2021["Absolute error"].values
    < soi_from_tc_puf_2021["Absolute error"].values
)

pct_improved = soi_from_tmd_2021["Improved"].mean()

Markdown(
    f"**{pct_improved:.1%}** of SOI statistics improved over reweighting."
)

**58.1%** of SOI statistics improved over reweighting.

## SOI aggregates by variable

The below table shows the SOI statistics (and how TMD compares) for all SOI variables in aggregate (all-filer-statistics, total and counts).

In [5]:
# Show all rows

pd.set_option("display.max_rows", None)

soi_from_tmd_2021[soi_from_tmd_2021["Full population"]].sort_values(
    ["Variable"]
).drop(
    columns=[
        "Full population",
        "Filing status",
        "AGI lower bound",
        "AGI upper bound",
        "Taxable only",
    ]
)

,Variable,Count,Value,SOI Value,Error,Absolute error,Relative error,Absolute relative error,OK,Improved
2,adjusted_gross_income,False,1.473366e+13,14795614070000,-6.195162e+10,6.195162e+10,-0.004187,0.004187,True,True
190,business_net_losses,False,1.077823e+11,105580403000,2.201941e+09,2.201941e+09,0.020856,0.020856,True,True
192,business_net_losses,True,7.561762e+06,7546660,1.510163e+04,1.510163e+04,0.002001,0.002001,True,True
262,business_net_profits,False,5.578561e+11,517081772000,4.077436e+10,4.077436e+10,0.078855,0.078855,False,True
264,business_net_profits,True,2.181236e+07,21105685,7.066768e+05,7.066768e+05,0.033483,0.033483,True,True
334,capital_gains_distributions,False,2.458712e+10,23889533000,6.975849e+08,6.975849e+08,0.029200,0.029200,True,False
336,capital_gains_distributions,True,4.688608e+06,4505544,1.830636e+05,1.830636e+05,0.040631,0.040631,True,False
406,capital_gains_gross,False,1.978673e+12,2048795356000,-7.012234e+10,7.012234e+10,-0.034226,0.034226,True,True
408,capital_gains_gross,True,1.854735e+07,20497375,-1.950028e+06,1.950028e+06,-0.095135,0.095135,False,True
478,capital_gains_losses,False,1.599024e+10,16241889000,-2.516539e+08,2.516539e+08,-0.015494,0.015494,True,True


## SALT distributions

In [6]:
soi_from_tmd_2021[soi_from_tmd_2021.Variable == "state_and_local_tax_deductions"].sort_values(["Count", "AGI lower bound"])

,Variable,Filing status,AGI lower bound,AGI upper bound,Count,Taxable only,Full population,Value,SOI Value,Error,Absolute error,Relative error,Absolute relative error,OK,Improved
1893,state_and_local_tax_deductions,All,-inf,inf,False,False,True,3.166909e+11,362507801000,-4.581686e+10,4.581686e+10,-0.126389,0.126389,False,False
1894,state_and_local_tax_deductions,All,-inf,inf,False,True,False,3.051561e+11,352542556000,-4.738650e+10,4.738650e+10,-0.134414,0.134414,False,False
1897,state_and_local_tax_deductions,All,0.0,5000.0,False,False,False,4.240925e+08,336650000,8.744255e+07,8.744255e+07,0.259743,0.259743,True,True
1899,state_and_local_tax_deductions,All,5000.0,10000.0,False,False,False,2.786683e+08,380502000,-1.018337e+08,1.018337e+08,-0.267630,0.267630,True,True
1901,state_and_local_tax_deductions,All,10000.0,15000.0,False,False,False,4.228359e+08,581150000,-1.583141e+08,1.583141e+08,-0.272415,0.272415,True,True
1903,state_and_local_tax_deductions,All,15000.0,20000.0,False,False,False,5.540467e+08,821023000,-2.669763e+08,2.669763e+08,-0.325175,0.325175,True,False
1905,state_and_local_tax_deductions,All,20000.0,25000.0,False,False,False,3.636382e+08,811022000,-4.473838e+08,4.473838e+08,-0.551630,0.551630,True,False
1907,state_and_local_tax_deductions,All,25000.0,30000.0,False,False,False,3.577957e+08,835417000,-4.776213e+08,4.776213e+08,-0.571716,0.571716,True,False
1909,state_and_local_tax_deductions,All,30000.0,35000.0,False,False,False,6.735870e+08,1209086000,-5.354990e+08,5.354990e+08,-0.442896,0.442896,True,False
1911,state_and_local_tax_deductions,All,35000.0,40000.0,False,False,False,7.754811e+08,1372777000,-5.972959e+08,5.972959e+08,-0.435100,0.435100,True,False
